# Warsztat 2: Ingestion Pipeline - COPY INTO & Auto Loader

**Cel warsztatu:**
- Implementacja batch ingestion używając COPY INTO
- Konfiguracja Auto Loader dla streaming ingestion
- Obsługa różnych formatów plików (CSV, JSON, Parquet)
- Monitorowanie i zarządzanie pipeline'ami ingestion

**Czas:** 90 minut

---

## 📚 Inicjalizacja środowiska

In [ ]:
%run ../../00_setup

## Część 1: COPY INTO - Batch Ingestion

### Zadanie 1.1: Ingestion plików CSV

**Instrukcje:**
1. Przygotuj tabelę docelową `bronze_customers_batch`
2. Użyj `COPY INTO` do załadowania danych z `customers.csv`
3. Zweryfikuj liczbę załadowanych rekordów

In [ ]:
# TODO: Utwórz tabelę docelową
spark.sql(f"""
 CREATE TABLE IF NOT EXISTS {CATALOG}.{SCHEMA}.bronze_customers_batch (
 customer_id INT,
 name STRING,
 email STRING,
 city STRING,
 country STRING,
 _ingestion_timestamp TIMESTAMP
 )
 USING DELTA
 LOCATION '{____}/customers_batch' -- Uzupełnij BRONZE_PATH
""")

In [ ]:
# TODO: COPY INTO z pliku CSV
spark.sql(f"""
 ____ INTO {CATALOG}.{SCHEMA}.bronze_customers_batch
 FROM (
 SELECT 
 customer_id,
 name,
 email,
 city,
 country,
 current_timestamp() as _ingestion_timestamp
 FROM '{____}/____' -- Uzupełnij SOURCE_DATA_PATH i nazwę pliku
 )
 FILEFORMAT = ____ -- Uzupełnij format (CSV)
 FORMAT_OPTIONS (
 'header' = '____', -- Czy plik ma nagłówek?
 'inferSchema' = '____' -- Czy inferować schemat?
 )
 COPY_OPTIONS (
 'mergeSchema' = '____' -- Czy mergować schemat?
 )
""")

In [ ]:
# Weryfikacja
spark.sql(f"""
 SELECT COUNT(*) as total_records 
 FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""").show()

### Zadanie 1.2: Ingestion plików JSON

**Instrukcje:**
1. Przygotuj tabelę `bronze_orders_batch`
2. Użyj `COPY INTO` do załadowania danych z `orders_batch.json`
3. Obsłuż zagnieżdżoną strukturę JSON

In [ ]:
# TODO: Utwórz tabelę dla zamówień
spark.sql(f"""
 CREATE TABLE IF NOT EXISTS {CATALOG}.{SCHEMA}.bronze_orders_batch (
 order_id INT,
 customer_id INT,
 order_date DATE,
 total_amount DOUBLE,
 status STRING,
 _ingestion_timestamp TIMESTAMP
 )
 USING ____ -- Uzupełnij format (DELTA)
 LOCATION '{____}/orders_batch' -- Uzupełnij ścieżkę
""")

In [ ]:
# TODO: COPY INTO z pliku JSON
spark.sql(f"""
 COPY INTO {CATALOG}.{SCHEMA}.bronze_orders_batch
 FROM (
 SELECT 
 order_id,
 customer_id,
 TO_DATE(order_date) as order_date,
 total_amount,
 status,
 current_timestamp() as _ingestion_timestamp
 FROM '{____}/____' -- Uzupełnij ścieżkę do JSON
 )
 FILEFORMAT = ____ -- Uzupełnij format
""")

In [ ]:
# Weryfikacja
spark.sql(f"""
 SELECT * FROM {CATALOG}.{SCHEMA}.bronze_orders_batch LIMIT 10
""").show()

### Zadanie 1.3: Ingestion plików Parquet

**Instrukcje:**
1. Przygotuj tabelę `bronze_products_batch`
2. Użyj `COPY INTO` do załadowania danych z `products.parquet`
3. Dodaj kolumnę z metadanymi pliku źródłowego

In [ ]:
# TODO: Utwórz tabelę dla produktów
spark.sql(f"""
 CREATE TABLE IF NOT EXISTS {CATALOG}.{SCHEMA}.bronze_products_batch (
 product_id INT,
 product_name STRING,
 category STRING,
 price DOUBLE,
 stock_quantity INT,
 _source_file STRING,
 _ingestion_timestamp TIMESTAMP
 )
 USING DELTA
 LOCATION '{____}/products_batch'
""")

In [ ]:
# TODO: COPY INTO z pliku Parquet
spark.sql(f"""
 COPY INTO {CATALOG}.{SCHEMA}.bronze_products_batch
 FROM (
 SELECT 
 product_id,
 product_name,
 category,
 price,
 stock_quantity,
 ____ as _source_file, -- Użyj _metadata.file_path
 current_timestamp() as _ingestion_timestamp
 FROM '{____}/____' -- Uzupełnij ścieżkę do Parquet
 )
 FILEFORMAT = ____
""")

In [ ]:
# Weryfikacja
spark.sql(f"""
 SELECT product_id, product_name, category, _source_file 
 FROM {CATALOG}.{SCHEMA}.bronze_products_batch 
 LIMIT 10
""").show(truncate=False)

### Zadanie 1.4: Idempotentność - Ponowne uruchomienie COPY INTO

**Instrukcje:**
1. Uruchom ponownie `COPY INTO` dla tej samej tabeli
2. Zweryfikuj że dane nie zostały zduplikowane
3. Sprawdź historię operacji `COPY INTO`

In [ ]:
# Sprawdź liczbę rekordów przed ponownym COPY INTO
before_count = spark.sql(f"""
 SELECT COUNT(*) as count 
 FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""").collect()[0]["count"]

print(f"Liczba rekordów przed ponownym COPY INTO: {before_count}")

In [ ]:
# TODO: Ponowne wykonanie COPY INTO
spark.sql(f"""
 COPY INTO {CATALOG}.{SCHEMA}.bronze_customers_batch
 FROM (
 SELECT 
 customer_id,
 name,
 email,
 city,
 country,
 current_timestamp() as _ingestion_timestamp
 FROM '{SOURCE_DATA_PATH}/customers.csv'
 )
 FILEFORMAT = CSV
 FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
""")

In [ ]:
# Weryfikacja idempotentności
after_count = spark.sql(f"""
 SELECT COUNT(*) as count 
 FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""").collect()[0]["count"]

print(f"Liczba rekordów po ponownym COPY INTO: {after_count}")
print(f"Czy dane zostały zduplikowane? {before_count != after_count}")

---

## Część 2: Auto Loader - Streaming Ingestion

### Zadanie 2.1: Konfiguracja Auto Loader dla CSV

**Instrukcje:**
1. Przygotuj lokalizację checkpointu
2. Użyj `.format("cloudFiles")` do utworzenia streaming read
3. Skonfiguruj schema inference i evolution
4. Zapisz stream do tabeli `bronze_customers_stream`

In [ ]:
# TODO: Streaming read z Auto Loader
customers_stream = (
 spark.readStream
 .format("____") # Uzupełnij format (cloudFiles)
 .option("cloudFiles.format", "____") # Format plików źródłowych (csv)
 .option("cloudFiles.schemaLocation", f"{CHECKPOINT_PATH}/____") # Checkpoint dla schematu
 .option("header", "true")
 .load(f"{SOURCE_DATA_PATH}/customers.csv")
)

In [ ]:
# TODO: Dodaj kolumny metadata
from pyspark.sql.functions import current_timestamp, input_file_name

customers_enriched = (
 customers_stream
 .withColumn("_ingestion_timestamp", ____) # Dodaj timestamp
 .withColumn("_source_file", ____) # Dodaj nazwę pliku źródłowego
)

In [ ]:
# TODO: Zapisz stream do tabeli Delta
query_customers = (
 customers_enriched.writeStream
 .format("____") # Uzupełnij format
 .outputMode("____") # Tryb zapisu (append)
 .option("checkpointLocation", f"{____}/customers_stream") # Checkpoint
 .option("mergeSchema", "____") # Schema evolution
 .table(f"{CATALOG}.{SCHEMA}.bronze_customers_stream")
)

In [ ]:
# Weryfikacja streamu
import time
time.sleep(10) # Poczekaj na przetworzenie

spark.sql(f"""
 SELECT COUNT(*) as total_records 
 FROM {CATALOG}.{SCHEMA}.bronze_customers_stream
""").show()

### Zadanie 2.2: Auto Loader dla JSON z Schema Hints

**Instrukcje:**
1. Użyj Auto Loader do odczytu `orders_batch.json`
2. Dodaj schema hints dla kolumn z konkretnymi typami
3. Skonfiguruj rescue data column dla niepoprawnych rekordów

In [ ]:
# TODO: Auto Loader z schema hints
orders_stream = (
 spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "____") # Format JSON
 .option("cloudFiles.schemaLocation", f"{CHECKPOINT_PATH}/orders_schema")
 .option("cloudFiles.schemaHints", "____") # Podpowiedź: "order_date DATE, total_amount DOUBLE"
 .option("cloudFiles.rescuedDataColumn", "____") # Kolumna dla rescue data
 .load(f"{____}/orders_batch.json") # Uzupełnij ścieżkę
)

In [ ]:
# TODO: Zapisz stream
query_orders = (
 orders_stream.writeStream
 .format("delta")
 .outputMode("____") # Tryb append
 .option("checkpointLocation", f"{CHECKPOINT_PATH}/____") # Checkpoint
 .table(f"{CATALOG}.{SCHEMA}.bronze_orders_stream")
)

In [ ]:
# Weryfikacja
time.sleep(10)
spark.sql(f"""
 SELECT * FROM {CATALOG}.{SCHEMA}.bronze_orders_stream LIMIT 10
""").show()

### Zadanie 2.3: Monitoring streaming queries

**Instrukcje:**
1. Wyświetl aktywne streaming queries
2. Sprawdź status i ostatni progress każdego query
3. Pobierz metryki: liczba przetworzonych rekordów, batch duration

In [ ]:
# TODO: Wyświetl aktywne streamy
active_streams = spark.streams.____ # Uzupełnij metodę (active)

print(f"Liczba aktywnych streamów: {len(active_streams)}")
for stream in active_streams:
 print(f"\nStream ID: {stream.id}")
 print(f"Name: {stream.name}")
 print(f"Status: {stream.status}")

In [ ]:
# TODO: Sprawdź ostatni progress
if len(active_streams) > 0:
 last_progress = active_streams[0].____ # Uzupełnij metodę (lastProgress)
 
 if last_progress:
 print(f"Batch ID: {last_progress['batchId']}")
 print(f"Przetworzone rekordy: {last_progress['numInputRows']}")
 print(f"Czas przetwarzania: {last_progress['batchDuration']} ms")

### Zadanie 2.4: Zatrzymanie streaming queries

**Instrukcje:**
1. Zatrzymaj wszystkie aktywne streaming queries
2. Zweryfikuj że wszystkie streamy są zatrzymane

In [ ]:
# TODO: Zatrzymaj wszystkie streamy
for stream in spark.streams.active:
 print(f"Zatrzymuję stream: {stream.name}")
 stream.____() # Uzupełnij metodę (stop)

print("\nWszystkie streamy zatrzymane!")

In [ ]:
# Weryfikacja
print(f"Liczba aktywnych streamów: {len(spark.streams.active)}")

---

## 📊 Część 3: Porównanie COPY INTO vs Auto Loader

### Zadanie 3.1: Analiza wydajności

**Instrukcje:**
1. Porównaj liczbę rekordów załadowanych przez COPY INTO vs Auto Loader
2. Sprawdź historię operacji dla obu metod
3. Zidentyfikuj przypadki użycia dla każdej metody

In [ ]:
# Porównanie liczby rekordów
copy_into_count = spark.sql(f"""
 SELECT 'COPY INTO' as method, COUNT(*) as records 
 FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""")

auto_loader_count = spark.sql(f"""
 SELECT 'Auto Loader' as method, COUNT(*) as records 
 FROM {CATALOG}.{SCHEMA}.bronze_customers_stream
""")

copy_into_count.union(auto_loader_count).show()

In [ ]:
# TODO: Historia operacji COPY INTO
spark.sql(f"""
 ____ HISTORY {CATALOG}.{SCHEMA}.bronze_customers_batch
""").select("version", "operation", "operationMetrics").show(truncate=False)

In [ ]:
# TODO: Historia operacji Auto Loader
spark.sql(f"""
 DESCRIBE HISTORY {CATALOG}.{SCHEMA}.bronze_customers_stream
""").select("version", "operation", "operationMetrics").show(truncate=False)

---

## Podsumowanie warsztatu

**Zrealizowane cele:**
- Implementacja batch ingestion z COPY INTO
- Konfiguracja Auto Loader dla streaming ingestion
- Obsługa różnych formatów (CSV, JSON, Parquet)
- Monitoring i zarządzanie pipeline'ami

**Kiedy użyć COPY INTO:**
- Batch processing z określonym harmonogramem
- Znana i stabilna struktura danych
- Potrzeba kontroli nad procesem ładowania
- Idempotentność out-of-the-box

**Kiedy użyć Auto Loader:**
- Near real-time processing
- Schema evolution i automatyczna inferencja
- Ciągłe monitorowanie nowych plików
- Skalowalność i efektywność kosztowa

---

## 🧹 Cleanup (opcjonalnie)